<a href="https://colab.research.google.com/github/GRARLBD/FIA2023-TPI/blob/main/FIA2023_TPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages('lubridate')
install.packages('ggplot2')
install.packages('ggdist')

In [ ]:
library(lubridate)
library(ggplot2)
library(ggdist)
library(dplyr)

In [5]:
df_princess<- read.csv("/content/princess.csv")
df_wine <- read.csv("/content/wine.csv")

In [ ]:


table(df_princess$Cured)

In [ ]:
install.packages("e1071")#svm
install.packages("randomForest") #randomForest
install.packages("clock") #necesaria para instalar 'caret'
install.packages("caret") #para el particionado del df
install.packages("fastDummies") #para realizar el casteo (one-hot encoding)

In [ ]:
library(e1071)
library(randomForest)
library(dplyr)
library(caret)
library(reshape2)
library(fastDummies)

In [ ]:
head(df_wine) # MOSTRAMOS LA PRIMERAS FILAS


In [ ]:
df_label_bad <- df_wine[df_wine$quality == 'bad', ] # Creamos un df con las filas con etiqueta 0
sampled_df_label_bad <- df_label_bad[sample(nrow(df_label_bad), 10), ]

# Creamos un subconjunto con las filas con label = 1
df_label_good <- df_wine[df_wine$quality == 'good', ]
sampled_df_label_good <- df_label_good[sample(nrow(df_label_good), 10), ]
# Combinamos los subconjuntos en un DataFrame balanceado
df_balanceado160SVM <- rbind(sampled_df_label_bad, sampled_df_label_good)

df_balanceado160SVM

In [55]:
set.seed(2023) #seteamos la semilla para que el experimento sea reproducible
t.ids <- createDataPartition(df_wine$quality, p =0.7, list=F) #particionamos el df en 70% para entrenamiento y 30% para prueba


In [57]:
# Asegúrate de que la columna 'quality' sea de tipo factor
df_wine$quality <- as.factor(df_wine$quality)

# Entrenar el modelo de Random Forest
modelo_rf <- randomForest(quality ~ ., data = df_wine[t.ids, ])

# Realizar predicciones
prediccion_rf <- predict(modelo_rf, newdata = df_wine[-t.ids, ])
matriz_confusion <- table(prediccion_rf, df_wine$quality[-t.ids], dnn = c('Real' ,'Predicho'))



In [ ]:
# Evaluación del modelo u otros procedimientos
matriz_confusion <- table(prediccion_rf, df_wine$quality[-t.ids], dnn = c('Real' ,'Predicho'))
# Cálculo de métricas a partir de la matriz de confusión
library(caret)

# Convertir la matriz de confusión en un objeto de tipo ConfusionMatrix para calcular métricas
confusion_matrix <- confusionMatrix(matriz_confusion)

# Obtener la precisión del modelo
precision <- confusion_matrix$overall['Accuracy']
cat("Precisión (Accuracy):", precision, "\n\n\n")
confusion_matrix


In [64]:
#SVM
# Asegúrate de que la columna 'quality' sea de tipo factor
df_wine$quality <- as.factor(df_wine$quality)

modelo <- svm(quality ~ ., data = df_wine[t.ids, ]) #entrenamiento
prediccion <- predict(modelo, df_wine[-t.ids, ]) #prueba
matriz_confusion <- table(df_wine$quality[-t.ids], prediccion, dnn = c('Real', 'Predicho')) #creamos una matriz de confusión

# Cálculo de métricas a partir de la matriz de confusión
library(caret)

# Convertir la matriz de confusión en un objeto de tipo ConfusionMatrix para calcular métricas
confusion_matrix <- confusionMatrix(matriz_confusion)

# Obtener la precisión del modelo
precision <- confusion_matrix$overall['Accuracy']
cat("Precisión (Accuracy):", precision, "\n\n\n")
confusion_matrix

Precisión (Accuracy): 0.743215 




Confusion Matrix and Statistics

      Predicho
Real   bad good
  bad  160   63
  good  60  196
                                          
               Accuracy : 0.7432          
                 95% CI : (0.7016, 0.7818)
    No Information Rate : 0.5407          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.4835          
                                          
 Mcnemar's Test P-Value : 0.8569          
                                          
            Sensitivity : 0.7273          
            Specificity : 0.7568          
         Pos Pred Value : 0.7175          
         Neg Pred Value : 0.7656          
             Prevalence : 0.4593          
         Detection Rate : 0.3340          
   Detection Prevalence : 0.4656          
      Balanced Accuracy : 0.7420          
                                          
       'Positive' Class : bad             
                                          